In [ ]:
# 将PaddleDetection套件进行解压，路径看具体情况。
# (为了保险起见，将套件保存至work/路径下，千万不要保存在data/下以及这两个文件夹外，作者有过惨痛的经历教训，不希望大家再出现此惨状。)

!unzip -oq /home/aistudio/data/data112279/PaddleDetection-release-0.5.zip -d work/

In [ ]:
# 将PaddleDetection套件进行改名（改名记得写上套件路径及改名后期望的路径）

!mv work/PaddleDetection-release-0.5 work/PaddleDetection

In [ ]:
!unzip -o data/data85839/RBC.zip -d work/PaddleDetection/dataset/

In [ ]:
!mv work/PaddleDetection/dataset/RBC work/PaddleDetection/dataset/test_det

In [ ]:
import os 
import random

# 类别数量 
file_saved = []   # 保存数据

random.seed(2021)

# voc数据路径问题
# 根目录信息，子目录信息，files_img--该文件夹下的文件名称
for _, _, files_img in os.walk('work/PaddleDetection/dataset/test_det/JPEGImages'):
    random.shuffle(files_img)

    for _, _, files_xml in os.walk('work/PaddleDetection/dataset/test_det/Annotations'):
        # indexs = 0
        # 1.jpg
        # 1.xml
        for i in range(len(files_img)):  # 遍历图片文件--一张一张的
            for j in range(len(files_xml)):
                # 匹配，与图片前缀名称一致的xml文件
                # 前缀是否一致
                if files_img[i][:-4] == files_xml[j][:-4]:
                    # 图片的相对路径 + 空格 + 标注文件的相对路径 + '\n'
                    # jpeg, img -- join -> jpeg/img
                    # JPEGImages/files_img[i]
                    file_maked = os.path.join('JPEGImages', files_img[i]) + ' ' + os.path.join('Annotations', files_xml[j])  + '\n'
                    
                    file_saved.append(file_maked)          # 每一个类别放在对应的缓存空间中
                    break

# example: 图片的相对路径 + 空格 + 标注文件的相对路径 + '\n'

# 训练集的划分
# 训练集占80%的数据
# 验证集/评估数据集：1-80% = 20%
Train_percent = 0.8

# train.txt保存
with open('work/PaddleDetection/dataset/test_det/train.txt', 'w') as f:
    # int(Train_percent * len(file_saved))
    # final_index = int(len(file_saved)*Train_percent) - 1
    f.writelines(file_saved[:int(len(file_saved)*Train_percent)])  # 写入多行数据
    print('train.txt Has Writed {0} records!'.format(len(file_saved[:int(len(file_saved)*Train_percent)])))
    
# eval.txt保存
with open('work/PaddleDetection/dataset/test_det/eval.txt', 'w') as f:
    # final_index + 1 == int(len(file_saved)*Train_percent)
    f.writelines(file_saved[int(len(file_saved)*Train_percent):])
    print('eval.txt Has Writed {0} records!'.format(len(file_saved[int(len(file_saved)*Train_percent):])))

train.txt Has Writed 274 records!
eval.txt Has Writed 69 records!


In [ ]:
%cd work/PaddleDetection/
!pip install -r requirements.txt

**yolov3_darknet_voc.yml**
```
architecture: YOLOv3
use_gpu: true
max_iters: 70000
log_iter: 20
save_dir: output
snapshot_iter: 2000
metric: VOC
map_type: 11point
pretrain_weights: https://paddle-imagenet-models-name.bj.bcebos.com/DarkNet53_pretrained.tar
weights: output/yolov3_darknet_voc/model_final
num_classes: 1
use_fine_grained_loss: false

YOLOv3:
  backbone: DarkNet
  yolo_head: YOLOv3Head

DarkNet:
  norm_type: sync_bn
  norm_decay: 0.
  depth: 53

YOLOv3Head:
  anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
  anchors: [[10, 13], [16, 30], [33, 23],
            [30, 61], [62, 45], [59, 119],
            [116, 90], [156, 198], [373, 326]]
  norm_decay: 0.
  yolo_loss: YOLOv3Loss
  nms:
    background_label: -1
    keep_top_k: 100
    nms_threshold: 0.45
    nms_top_k: 1000
    normalized: false
    score_threshold: 0.01

YOLOv3Loss:
  ignore_thresh: 0.7
  label_smooth: false

LearningRate:
  base_lr: 0.00025
  schedulers:
  - !PiecewiseDecay
    gamma: 0.1
    milestones:
    - 55000
    - 62000
  - !LinearWarmup
    start_factor: 0.
    steps: 1000

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.0005
    type: L2

_READER_: 'yolov3_reader.yml'
TrainReader:
  inputs_def:
    fields: ['image', 'gt_bbox', 'gt_class', 'gt_score']
    num_max_boxes: 50
  dataset:
    !VOCDataSet
    dataset_dir: dataset/test_det
    anno_path: train.txt
    use_default_label: false
    with_background: false

EvalReader:
  inputs_def:
    fields: ['image', 'im_size', 'im_id', 'gt_bbox', 'gt_class', 'is_difficult']
    num_max_boxes: 50
  dataset:
    !VOCDataSet
    dataset_dir: dataset/test_det
    anno_path: eval.txt
    use_default_label: false
    with_background: false

TestReader:
  dataset:
    !ImageFolder
    use_default_label: false
    with_background: false

```

**yolov3_reader.yml**
```
TrainReader:
  inputs_def:
    fields: ['image', 'gt_bbox', 'gt_class', 'gt_score']
    num_max_boxes: 50
  dataset:
    !COCODataSet
      image_dir: dataset/test_det
      anno_path: train.txt
      dataset_dir: dataset/test_det
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
      with_mixup: True
    - !MixupImage
      alpha: 1.5
      beta: 1.5
    - !ColorDistort {}
    - !RandomExpand
      fill_value: [123.675, 116.28, 103.53]
    - !RandomCrop {}
    - !RandomFlipImage
      is_normalized: false
    - !NormalizeBox {}
    - !PadBox
      num_max_boxes: 50
    - !BboxXYXY2XYWH {}
  batch_transforms:
  - !RandomShape
    sizes: [320, 352, 384, 416, 448, 480, 512, 544, 576, 608]
    random_inter: True
  - !NormalizeImage
    mean: [0.485, 0.456, 0.406]
    std: [0.229, 0.224, 0.225]
    is_scale: True
    is_channel_first: false
  - !Permute
    to_bgr: false
    channel_first: True
  # Gt2YoloTarget is only used when use_fine_grained_loss set as true,
  # this operator will be deleted automatically if use_fine_grained_loss
  # is set as false
  - !Gt2YoloTarget
    anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    anchors: [[10, 13], [16, 30], [33, 23],
              [30, 61], [62, 45], [59, 119],
              [116, 90], [156, 198], [373, 326]]
    downsample_ratios: [32, 16, 8]
  batch_size: 8
  shuffle: true
  mixup_epoch: 250
  drop_last: true
  worker_num: 8
  bufsize: 16
  use_process: true


EvalReader:
  inputs_def:
    fields: ['image', 'im_size', 'im_id']
    num_max_boxes: 50
  dataset:
    !COCODataSet
      image_dir: dataset/test_det
      anno_path: eval.txt
      dataset_dir: dataset/test_det
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
    - !ResizeImage
      target_size: 608
      interp: 2
    - !NormalizeImage
      mean: [0.485, 0.456, 0.406]
      std: [0.229, 0.224, 0.225]
      is_scale: True
      is_channel_first: false
    - !PadBox
      num_max_boxes: 50
    - !Permute
      to_bgr: false
      channel_first: True
  batch_size: 1
  drop_empty: false
  worker_num: 8
  bufsize: 16

TestReader:
  inputs_def:
    image_shape: [3, 608, 608]
    fields: ['image', 'im_size', 'im_id']
  dataset:
    !ImageFolder
      anno_path: dataset/test_det/label_list.txt
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
    - !ResizeImage
      target_size: 608
      interp: 2
    - !NormalizeImage
      mean: [0.485, 0.456, 0.406]
      std: [0.229, 0.224, 0.225]
      is_scale: True
      is_channel_first: false
    - !Permute
      to_bgr: false
      channel_first: True
  batch_size: 1

```

In [ ]:
%cd work/PaddleDetection/
!python tools/train.py -c configs/yolov3_darknet_voc.yml\
--eval\
--use_vdl True

In [2]:
%cd work/PaddleDetection/
!python tools/infer.py -c configs/yolov3_darknet_voc.yml\
-o weights='output/yolov3_darknet_voc/model_final.pdparams'\
--infer_dir 'dataset/test_det/JPEGImages'\
--output_dir 'output'\
--draw_threshold 0.1\
# --save_txt True

In [1]:
!python tools/export_model.py -c configs/yolov3_darknet_voc.yml --output_dir=./inference_model \
 -o weights=output/yolov3_darknet_voc/best_model

config.json<br>
```
{
	"network_type":"YOLOV3",

	"model_file_name":"__model__",
	"params_file_name":"__params__",

	"labels_file_name":"label_list.txt",

	"format":"RGB",
	"input_width":608,
	"input_height":608,
		
	"mean":[123.675, 116.28, 103.53],
    "scale":[0.0171248, 0.017507, 0.0174292],
	"threshold":0.01
}
```
image.json
```
{
    "model_config": "../../../res/models/detection/yolov3",
    "input": {
        "type": "image",
        "path": "../../../res/models/detection/yolov3/img/RBC.jpg"
    },
    "debug": {
        "display_enable": true,
        "predict_log_enable": true,
        "predict_time_log_enable": false
    }
}
```

部署代码：
```
cd /root/workspace/PaddleLiteDemo/C++/build
./detection ../../configs/detection/yolov3/image.json
```

3. 主动式miniDP公转HDMI母转换线：来连接EdgeBoard和HDMI显示器。<br>
参考链接：https://aim.baidu.com/product/649bd2ab-65ef-480d-b1bf-2ba8f10059ab<br>
建议使用eb的原产线，因为市场上的线质量参差不齐。

<img style="display: block; margin: 0 auto;" 
    src="https://ai-studio-static-online.cdn.bcebos.com/28fb1ffdb2384c5da2b180074e5e19a9186a86dd54824a1ab8b81e9c37fb9200" width = "50%" height = "50%" />

## 调用摄像头

### 1. 安装驱动（在eb终端输入命令）

```
insmod /root/workspace/driver/driver/ch341.ko
```
```
insmod /root/workspace/driver/driver/usbserial.ko
```
```
insmod /root/workspace/driver/driver/fpgadrv.ko
```

### 2. 测试摄像头（在eb终端输入命令）
摄像头参考链接：https://m.tb.cn/h.fRe4a6u?sm=0deb57<br>
本人使用的是usb工业摄像头jpg/YUY2高速 HF877_3mm_90度无畸变60帧。

<img style="display: block; margin: 0 auto;" 
    src="https://ai-studio-static-online.cdn.bcebos.com/1120088a4c934955a50a73292e2ede9b9083de0305734ca3abe71676cc6147b8" width = "50%" height = "50%" />

 

测试脚本：test_onecamera.py（由郑博培男神贡献的脚本，在此感谢郑博培男神，此脚本我已上传至文件中，大家有需要的可以自行下载）。<br>

```
python3 test_camera.py
```

能看到显示屏上有个蓝色的圈圈以及eb终端上有test！以及坐标，就说明摄像头测试成功。

<img style="display: block; margin: 0 auto;" 
    src="https://ai-studio-static-online.cdn.bcebos.com/88ded36da6ea4d7e8380a68cc93d295a9533f924efa24187b5628ea4fd9b6c1d" width = "50%" height = "50%" />

测试完，按Ctrl+C退出。

有点遗憾的是，很抱歉没完成摄像头预测，没能帮到大家。希望有大佬可以解决这个摄像头预测。


model、params(前两个文件名前后都有__,由于打上去会变成字体加粗，就省去了）、detectionResult、config.json、image.json、label.txt文件都上传至项目文件中，大家可以按照步骤自己训练、调参，也可以体验下本项目训练出来的模型参数与在板子上预测出来的结果图。

# 七、总结与升华

部署首先得把模型训练正常，并且正确导出模型，模型要能用Netron[https://netron.app/](https://netron.app/)（因此网站用特殊工具才能打开，所以我将PC端启动程序放置在项目文件中）打开，不然可能模型错误。拿到板子前要先测试，看看tf卡是不是好的。板子要注意用1.8.x版本训练，比较稳定，并且要看好板子支持的部署模型，之前上课期间因不知eb不支持ppyolo模型。后来知道了，就换成了yolov3。并且我也由2.1的两个模型换成了1.8.x来运行模型。重要东西一定要保存在work/目录下，防止某一天重启环境，notebook桌面里的文件丢失。特别是训练特别久的模型。模型导出后最好不要改模型文件的名字，以免部署失败。要注意image.json下的文件路径。希望这个项目可以帮助到大家。本以为这个项目要完不成了，突然部署的最后一天来了希望。这真是20岁第一天的好兆头。

# 个人简介
请大家多多支持。欢迎fork、喜欢、分享、评论。本人能力有限，经验不足，若有不足，欢迎指正，谢谢大家！<br>
> 姓名：林勇

> 学校：西安科技大学高新学院

> 学历：本科二年级在读

> 专业：智能科学与技术

> 擅长领域：cv（目标检测）

> AI Studio链接：我在AI Studio上获得白银等级，点亮6个徽章，来互关呀~<br>
[https://aistudio.baidu.com/aistudio/personalcenter/thirdview/691883](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/691883)

> github链接：[https://github.com/FZSWJLY](https://github.com/FZSWJLY)

> qq：1431364191

> 微信：programmerxiaoyong
<img style="display: block; margin: 0 auto;" src="https://ai-studio-static-online.cdn.bcebos.com/5eaf05881cc441769bee9165e93f73473ac31a5ff65d44ada3c3343138cba405" width = "50%" height = "50%" />